In [2]:
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd

df_train = pd.read_csv('df_train.csv', index_col='row_id')
df_test = pd.read_csv('df_test.csv', index_col='row_id')
target_train = pd.read_csv('target_train.csv', index_col='row_id').squeeze()
target_test = pd.read_csv('target_test.csv', index_col='row_id').squeeze()

In [8]:
def smape_loss(y_true, y_pred):
    """
    Custom SMAPE loss function for LightGBM
    """
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

def xgb_smape_eval(y_pred, y_true):
    y_true = y_true.get_label()
    return 'SMAPE', smape_loss(y_true, y_pred)

def smape_cv(model, df_train, target_train):
    tscv = TimeSeriesSplit(n_splits=5)
    smape_list = []
    for _, (train_index, test_index) in enumerate(tscv.split(df_train), start=1):
        # Create training and validation datasets
        X_train = df_train.iloc[train_index]
        y_train = target_train.iloc[train_index]
        X_valid = df_train.iloc[test_index]
        y_valid = target_train.iloc[test_index]

        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=xgb_smape_eval, verbose=False)

        # Make predictions
        y_pred = np.exp(model.predict(X_valid)) * X_valid['microbusiness_density']
        y_valid = np.exp(y_valid) * X_valid['microbusiness_density']

        # Calculate SMAPE score
        smape = smape_loss(y_valid, y_pred)
        smape_list.append(smape)

    print(f'Mean SMAPE: {np.mean(smape_list):.4f}')
    return np.mean(smape_list)

def xgb_objective(trial, df_train, target_train):
    """
    Objective function for XGBoost optimization without rolling window cross-validation.
    """
    params = {
        'n_estimators': 200,
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'random_state': 42,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'alpha': trial.suggest_float('alpha', 1e-2, 10.0),
        'lambda': trial.suggest_float('lambda', 1e-2, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 250),
    }

    model = xgb.XGBRegressor(**params)
    score = smape_cv(model, df_train, target_train)
    return score

In [10]:
import optuna

# Optimize the XGBoost model
xgb_study = optuna.create_study(direction='minimize')
xgb_study.optimize(lambda trial: xgb_objective(trial, df_train, target_train), n_trials=20)

# Print the best trial for XGBoost
print(f"XGBoost best trial: {xgb_study.best_trial.value} with parameters {xgb_study.best_trial.params}")


[I 2023-04-14 21:22:14,607] A new study created in memory with name: no-name-6f57d486-3fe6-49f4-890f-e0572cb824b0
/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-04-14 21:22:35,883] Trial 0 finished with value: 1.803249470593744 and parameters: {'colsample_bytree': 0.15996327708571992, 'colsample_bynode': 0.2377195265350667, 'max_depth': 8, 'learning_rate': 0.04916567077570185, 'alpha': 4.247074663537053, 'lambda': 5.1522160099247, 'min_child_weight': 73}. Best is trial 0 with value: 1.803249470593744.


Mean SMAPE: 1.8032


[I 2023-04-14 21:23:02,881] Trial 1 finished with value: 1.8079803846530176 and parameters: {'colsample_bytree': 0.5246112284274684, 'colsample_bynode': 0.32254866065534993, 'max_depth': 6, 'learning_rate': 0.04041848464609895, 'alpha': 6.910955818510056, 'lambda': 8.201931571879708, 'min_child_weight': 59}. Best is trial 0 with value: 1.803249470593744.


Mean SMAPE: 1.8080


[I 2023-04-14 21:23:37,531] Trial 2 finished with value: 1.8374242055638423 and parameters: {'colsample_bytree': 0.8969775740037893, 'colsample_bynode': 0.5517303227364284, 'max_depth': 4, 'learning_rate': 0.06452171612080242, 'alpha': 2.0727376185425914, 'lambda': 2.7888406027341457, 'min_child_weight': 34}. Best is trial 0 with value: 1.803249470593744.


...

Mean SMAPE: 1.7978


[I 2023-04-14 21:30:56,736] Trial 18 finished with value: 1.8133464667972459 and parameters: {'colsample_bytree': 0.37588198210016677, 'colsample_bynode': 0.5150589517352075, 'max_depth': 5, 'learning_rate': 0.07410277178128263, 'alpha': 3.2256701142538433, 'lambda': 4.00769844613148, 'min_child_weight': 119}. Best is trial 17 with value: 1.797837432080779.


Mean SMAPE: 1.8133


[I 2023-04-14 21:31:20,094] Trial 19 finished with value: 1.823068282980378 and parameters: {'colsample_bytree': 0.20585844831985012, 'colsample_bynode': 0.6666972960081703, 'max_depth': 7, 'learning_rate': 0.030065556249230234, 'alpha': 5.608142891394996, 'lambda': 8.959388838705339, 'min_child_weight': 153}. Best is trial 17 with value: 1.797837432080779.


Mean SMAPE: 1.8231
XGBoost best trial: 1.797837432080779 with parameters {'colsample_bytree': 0.1855095282932457, 'colsample_bynode': 0.4767718115446537, 'max_depth': 7, 'learning_rate': 0.049571666682710215, 'alpha': 3.9080858816277497, 'lambda': 4.8380412070727985, 'min_child_weight': 117}


In [5]:
train_df = pd.read_csv('train.csv')
params = {'colsample_bytree': 0.1855095282932457, 'colsample_bynode': 0.4767718115446537, 'max_depth': 7, 'learning_rate': 0.049571666682710215, 'alpha': 3.9080858816277497, 'lambda': 4.8380412070727985, 'min_child_weight': 117}
xgb_model = xgb.XGBRegressor(**params)

In [6]:
xgb_model.fit(df_train, target_train)
y_pred = np.exp(xgb_model.predict(df_test)) * df_test.microbusiness_density

In [9]:
my_cfips = df_train['cfips'].unique()
train = train_df[train_df.cfips.isin(my_cfips)]
pred = y_pred.values
y_true = train[train.row_id.str.contains('2022-07-01')].microbusiness_density.values
print(smape_loss(y_true, pred))

2.0149112484480667


In [ ]:
# clip_err = True

# if clip_err:
#     df_record = y_pred.merge(y_base, on = 'row_id', how='inner')
#     df_record = df_record.merge(y_true, on = 'row_id', how='inner')
#     def smape_(y_true, y_pred):
#         return 100  * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
#     base_err = df_record.apply(lambda x: smape_(x.y_true, x.y_base), axis=1)
#     pred_err = df_record.apply(lambda x: smape_(x.y_true, x.microbusiness_density), axis=1)
#     blacklist = base_err[(base_err + 1e-3) < pred_err].index
#     y_pred.iloc[blacklist] = y_base.iloc[blacklist]
#     avg_base_err = base_err.mean()
#     avg_pred_err = pred_err.mean()
#     print(avg_base_err, avg_pred_err)
#     df_sub = pd.read_csv('sample_submission.csv')
#     df_sub = df_sub.drop(columns='microbusiness_density')
#     df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
#     y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
#     merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
#     merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
#     # merged_df = merged_df.drop(columns='cfips')
# else:
#     df_sub = pd.read_csv('sample_submission.csv')
#     df_sub = df_sub.drop(columns='microbusiness_density')
#     df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
#     y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
#     merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
#     merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
#     # merged_df = merged_df.drop(columns='cfips')